In [121]:
import json
import uuid
import random
import pandas as pd
from datetime import datetime
import fhirclient.r3.models.patient as p
import fhirclient.r3.models.humanname as hn
import fhirclient.r3.models.observation as o
import fhirclient.r3.models.quantity as q
import fhirclient.r3.models.coding as c
import fhirclient.r3.models.codeableconcept as cc
import fhirclient.r3.models.fhirdate as dt
import fhirclient.r3.models.bundle as b
from faker import Faker

observation = o.Observation()
code = cc.CodeableConcept()
fake = Faker()

file = "loinc.csv"

df = pd.read_csv(file)
df

,Code,Display,ValueType,value,q.unit,q.system,q.code,cc.code,cc.system,cc.display
0,30180-4,Basophils/100 leukocytes in Blood,Quantity,1.0,percent,http://unitsofmeasure.org,%,NaN,NaN,NaN
1,26449-9,Eosinophils [#/volume] in Blood,Quantity,0.5,thousands per microliter,http://unitsofmeasure.org,10*3/uL,NaN,NaN,NaN
2,718-7,Hemoglobin [Mass/volume] in Blood,Quantity,15.0,gram per deciliter,http://unitsofmeasure.org,g/dL,NaN,NaN,NaN
3,26464-8,Leukocytes [#/volume] in Blood,Quantity,5.2,billion per liter,http://unitsofmeasure.org,10*9/L,NaN,NaN,NaN
4,40651-2,Metamyelocytes [Presence] in Blood,CodeableConcept,NaN,NaN,NaN,NaN,LA9633-4,http://loinc.org,Present
5,74398-9,Neutrophils [#/volume] in Cord blood,Quantity,7.9,billion per liter,http://unitsofmeasure.org,10*9/L,NaN,NaN,NaN


In [122]:
print(fake.first_name_female())
print(fake.last_name())

Victoria
Henderson


In [123]:
patient = p.Patient()
name = hn.HumanName()
given_name = fake.first_name()
family_name = fake.last_name()
name.family = family_name
name.given = [given_name, "Dash"]
name.text = given_name + " Dash " + family_name
name.use = 'official'
patient.name = [name]

patient.uuid = uuid.uuid4().urn
print(json.dumps(patient.as_json(), indent=4))
print(patient.uuid)

{
    "name": [
        {
            "family": "Martin",
            "given": [
                "Robert",
                "Dash"
            ],
            "text": "Robert Dash Martin",
            "use": "official"
        }
    ],
    "resourceType": "Patient"
}
urn:uuid:a543ba43-6e17-41e8-b35b-c08dc7ebeba0


In [125]:
bundle = b.Bundle()
bundle.type = 'transaction'
bundleEntry = b.BundleEntry()
bundleEntry.fullUrl = patient.uuid
bundleEntry.resource = patient
bundleEntryRequest = b.BundleEntryRequest()
bundleEntryRequest.method = "POST"
bundleEntryRequest.url = "Patient"
bundleEntry.request = bundleEntryRequest
bundle.entry = [bundleEntry]
print(json.dumps(bundle.as_json(), indent=4))

{
    "entry": [
        {
            "fullUrl": "urn:uuid:a543ba43-6e17-41e8-b35b-c08dc7ebeba0",
            "request": {
                "method": "POST",
                "url": "Patient"
            },
            "resource": {
                "name": [
                    {
                        "family": "Martin",
                        "given": [
                            "Robert",
                            "Dash"
                        ],
                        "text": "Robert Dash Martin",
                        "use": "official"
                    }
                ],
                "resourceType": "Patient"
            }
        }
    ],
    "type": "transaction",
    "resourceType": "Bundle"
}


In [114]:
observation.status = "final"
codecode = c.Coding()
codecode.system = 'http://loinc.org'

In [115]:
category = cc.CodeableConcept()
categorycoding = c.Coding()
categorycoding.system = "http://loinc.org"
categorycoding.code = "laboratory"
categorycoding.display = "Laboratory"
category.coding = [categorycoding]
observation.category = [category]

In [119]:
bundleEntry
effectiveDateTime = dt.FHIRDate()
valuequantity = q.Quantity()
for index, row in df.iterrows():
    if row['ValueType'] == 'Quantity':
        # for each LOINC code
        codecode.code = row['Code']
        codecode.display = row['Display']
        code.text = row['Display']
        code.coding = [codecode]
        valuequantity.unit = row['q.unit']
        valuequantity.system = row['q.system']
        valuequantity.code = row["q.code"]
        observation.code = code
        for i in range(random.randint(1, 10)):
            # make 1 to 10 examples
            valuequantity.value = row['value']
            observation.valueQuantity = valuequantity
            effectiveDateTime.date = fake.date_time_between(start_date='-5y', end_date='now')
            observation.effectiveDateTime = effectiveDateTime
            print(json.dumps(observation.as_json(), indent=4))
            

{
    "category": [
        {
            "coding": [
                {
                    "code": "laboratory",
                    "display": "Laboratory",
                    "system": "http://loinc.org"
                }
            ]
        }
    ],
    "code": {
        "coding": [
            {
                "code": "30180-4",
                "display": "Basophils/100 leukocytes in Blood",
                "system": "http://loinc.org"
            }
        ],
        "text": "Basophils/100 leukocytes in Blood"
    },
    "effectiveDateTime": "2021-04-24T09:22:57",
    "status": "final",
    "valueQuantity": {
        "code": "%",
        "system": "http://unitsofmeasure.org",
        "unit": "percent",
        "value": 1.0
    },
    "resourceType": "Observation"
}
{
    "category": [
        {
            "coding": [
                {
                    "code": "laboratory",
                    "display": "Laboratory",
                    "system": "http://loinc.org"
      

In [103]:
fake = Faker()

In [10]:
fake.name()

'Ellen Cummings'

In [12]:
fake.address()

'Unit 7111 Box 3875\nDPO AA 29937'

In [13]:
for _ in range(10):
    print(fake.name())

Mrs. Michelle Berg
James Meadows
Patrick Jordan
Amy Smith
Alexander Flores
Rachel Richards
Michael Shaw
Terry Wheeler
Kevin Adams
Jill Andrews


In [13]:
fake.date_time_between(start_date='-5y', end_date='now').isoformat()

'2019-11-07T08:18:09'

In [8]:
fakedate = fake.date_time_between(start_date='-5y', end_date='now').isoformat()
print(fakedate)

2020-06-30T07:45:12


In [37]:
edt = dt.FHIRDate()
# print(fakedate)
# edt.date = datetime.fromisoformat('2011-11-04T00:05:23+00:00')
edt = datetime.fromisoformat(fakedate)
print(edt)

2020-06-30 07:45:12


In [22]:
from datetime import datetime
my_date = datetime.now()
print(my_date.isoformat())

2021-07-07T16:39:30.106466


In [32]:
myfakedate = fake.date_time_between(start_date='-5y', end_date='now').isoformat()
myfakedate
datetime.fromisoformat(myfakedate).isoformat()

'2019-10-24T17:43:02'

In [26]:
from datetime import date
datetime.date.fromisoformat('2021-07-07T16:39:30.106466')

AttributeError: 'method_descriptor' object has no attribute 'fromisoformat'

In [77]:
import random
my_list = ['A'] * 20 + ['B'] * 20 + ['C'] * 60
random.choice(my_list)

'B'